In [1]:
import os
import sys
sys.path.append('..')

from src.financialqa.ingestion.parser import parse_pdfs, page_pdf_contents
from src.financialqa.ingestion.ingestionpipeline import IngestionPipeline
from src.financialqa.inference.inferencepipeline import InferencePipeline

from dotenv import load_dotenv
load_dotenv('../.env', override=True)

%load_ext autoreload
%autoreload 2

/anaconda/envs/finqa39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ingestion_pipeline = IngestionPipeline()

19:06:28 INFO src.financialqa.ingestion.ingestionpipeline line 77  Configured logging
19:06:28 INFO src.financialqa.ingestion.ingestionpipeline line 658  Set API variables
19:06:28 INFO src.financialqa.ingestion.ingestionpipeline line 601  Getting Azure Blob Storage container client 'messiah-test' from account 'eliasstorage2'
19:06:28 INFO src.financialqa.ingestion.ingestionpipeline line 618  Getting Azure AI Search client from service 'nlp-ai-search1'..
19:06:28 INFO src.financialqa.ingestion.ingestionpipeline line 631  Getting OpenAI embedding model 'text-embedding-3-small' from endpoint https://nonprodgaalaboai.openai.azure.com/..


In [3]:
pdf_paths = [
    os.path.join('../data', 'CIBC_Preview_Q1.pdf')
]

In [4]:
result_dicts = parse_pdfs(pdf_paths)

In [5]:
pdf_pages_dict = page_pdf_contents(result_dicts)

In [6]:
print(pdf_pages_dict.get('CIBC_Preview_Q1').get('pages').get(1).keys())
print(pdf_pages_dict.get('CIBC_Preview_Q1').get('pages').get(2).keys())
print(pdf_pages_dict.get('CIBC_Preview_Q1').get('pages').get(3).keys())

dict_keys(['text', 'tables', 'sectionHeading', 'footnote', 'pageFooter', 'figures'])
dict_keys(['text', 'tables', 'title', 'pageFooter', 'pageNumber', 'figures'])
dict_keys(['text', 'tables', 'pageNumber', 'pageFooter'])


In [7]:
ingestion_pipeline.crop_images_from_pdfs(
    pdf_pages_dict,
    save_as_jpg=True,
)

Processing figures on page 1 of CIBC_Preview_Q1...
Saving PDF 'CIBC_Preview_Q1' as JPEG to folder '../data/outputs/CIBC_Preview_Q1/jpg_outputs'.
Processing figures on page 2 of CIBC_Preview_Q1...
Saving PDF 'CIBC_Preview_Q1' as JPEG to folder '../data/outputs/CIBC_Preview_Q1/jpg_outputs'.
Processing figures on page 5 of CIBC_Preview_Q1...
Saving PDF 'CIBC_Preview_Q1' as JPEG to folder '../data/outputs/CIBC_Preview_Q1/jpg_outputs'.
Processing figures on page 6 of CIBC_Preview_Q1...
Saving PDF 'CIBC_Preview_Q1' as JPEG to folder '../data/outputs/CIBC_Preview_Q1/jpg_outputs'.
Processing figures on page 10 of CIBC_Preview_Q1...
Saving PDF 'CIBC_Preview_Q1' as JPEG to folder '../data/outputs/CIBC_Preview_Q1/jpg_outputs'.


In [8]:
ingestion_pipeline.generate_table_from_chart(
    pdf_pages_dict, 
    chart_model='chartvlm',
    chart_model_path='',
)

19:07:05 INFO src.financialqa.ingestion.ingestionpipeline line 198  Generating ChartVLM output for figure with JPG path: ../data/outputs/CIBC_Preview_Q1/jpg_outputs/figure_1_page_1.jpg...


/home/azureuser/cloudfiles/code/Users/ataeyme/projects/FinancialQA/src/financialqa/ingestion/models/ChartVLM-base/base_decoder


Legacy behavior is being used. The current behavior will be deprecated in version 5.0.0. In the new behavior, If both images and text are provided, image_processor is not a VQA processor, and `add_special_tokens` is unset, the default value of `add_special_tokens` will be changed to `False` when calling the tokenizer. To test the new behavior, set `legacy=False`as a processor call argument.


In [9]:
text_docs, table_docs, chart_docs = ingestion_pipeline.convert_paged_pdf_contents_to_docs(
    pdf_pages_dict,
    convert_tables=True,
    convert_charts=True,
)

19:11:55 INFO src.financialqa.ingestion.ingestionpipeline line 265  Converting extracted PDF page contents to Document objects..
19:11:55 INFO src.financialqa.ingestion.ingestionpipeline line 327  Missing chart table input for figure_1_page_2
19:11:55 INFO src.financialqa.ingestion.ingestionpipeline line 327  Missing chart table input for figure_1_page_5
19:11:55 INFO src.financialqa.ingestion.ingestionpipeline line 327  Missing chart table input for figure_1_page_6
19:11:55 INFO src.financialqa.ingestion.ingestionpipeline line 327  Missing chart table input for figure_1_page_10
19:11:55 INFO src.financialqa.ingestion.ingestionpipeline line 344  Created 20 text Documents, 17 table Documents, and 1 chart Documents


In [10]:
text_doc_chunks = ingestion_pipeline.chunk_docs(text_docs, chunk_size=400)
table_doc_chunks = ingestion_pipeline.chunk_docs(table_docs, chunk_size=400)
chart_doc_chunks = ingestion_pipeline.chunk_docs(chart_docs, chunk_size=400)

19:11:58 INFO src.financialqa.ingestion.ingestionpipeline line 357  Chunked 20 Documents to 36
19:11:58 INFO src.financialqa.ingestion.ingestionpipeline line 357  Chunked 17 Documents to 41
19:11:58 INFO src.financialqa.ingestion.ingestionpipeline line 357  Chunked 1 Documents to 1


In [11]:
for i, chunk in enumerate(text_doc_chunks):
# for i, chunk in enumerate(table_doc_chunks):
# for i, chunk in enumerate(chart_doc_chunks):
    metadata = chunk.metadata
    print(f"Page {metadata.get('page_num')} content:", chunk.page_content)
    print('Company name:', metadata.get('company_name'))
    print('Report quarter:', metadata.get('report_quarter'))
    print('Page titles:', metadata.get('page_titles'))
    print('Page headers:', metadata.get('page_headers'))
    print('Section headers:', metadata.get('section_headers'))
    print('Page footers:', metadata.get('page_footers'))
    print('')
    if i == 4:
        break

Page 1 content: CIBCO CIBC CAPITAL MARKETS EQUITY RESEARCH April 26, 2023 Earnings Revision Market Trends Favourable; IFRS 17 Still Presents Many Unknowns Lifecos have outperformed other financials by a good margin year to date. We are somewhat cautious heading into Q1 given the transition to IFRS 17 and lower-than-normal earnings visibility. However, we remain positive on the 12- to 24-month outlook versus banks based on expected EPS growth and capital optionality. The lifecos are trading at 1.3x P/BV on average (IFRS 17 adjusted), in line with the five-year trailing average of 1.3x. Our two Outperformer-rated names are IAG and GWO as we expect both to post strong earnings growth this year. Our investment thesis for the sector is summarized as follows: 1. IFRS 17 reduces near-term earnings visibility. Forecasting 2023 financials and providing a full scope analysis of Q1 results is significantly limited given that 2022 results pro forma IFRS 17 have not been provided by any lifeco. We 

In [12]:
ai_search_index = ingestion_pipeline.get_search_index(
    upload_docs=text_doc_chunks[:10],
    upload_docs_in_batches=True,
    batch_size=5,
    overwrite_index=True,
)

19:12:28 INFO src.financialqa.ingestion.ingestionpipeline line 372  Getting index 'finqa-test' from AI Search service 'nlp-ai-search1'..
19:12:28 INFO src.financialqa.ingestion.ingestionpipeline line 388  Found existing index 'finqa-test' in search service 'nlp-ai-search1'
19:12:28 INFO src.financialqa.ingestion.ingestionpipeline line 495  Overwriting index 'finqa-test'
19:12:30 INFO src.financialqa.ingestion.ingestionpipeline line 522  Attempting to upload batch of 5 Documents to index 'finqa-test'..
19:12:31 INFO src.financialqa.ingestion.ingestionpipeline line 535  Successfully uploaded batch of 5 Documents to index 'finqa-test' in 1 seconds
19:12:31 INFO src.financialqa.ingestion.ingestionpipeline line 522  Attempting to upload batch of 5 Documents to index 'finqa-test'..
19:12:32 INFO src.financialqa.ingestion.ingestionpipeline line 535  Successfully uploaded batch of 5 Documents to index 'finqa-test' in 1 seconds
19:12:32 INFO src.financialqa.ingestion.ingestionpipeline line 541 

In [13]:
inferencepipeline = InferencePipeline()

In [14]:
inferencepipeline.infer(
    query='What is the 5-Yr Average P/E?',
    company_name='CIBC',
    top_k=3,
)

Retrieved context: [Document(metadata={'id': 'MGVkM2QwMmMtMGJkNC00ZGUzLWJiOTMtYzdmNDJjMzlhOTBk', 'page_num': 2, 'pdf_name': 'CIBC_Preview_Q1', 'company_name': 'CIBC', 'report_quarter': 'Q1', 'page_titles': 'Industry Outlook', 'page_headers': '', 'section_headers': '', 'page_footers': 'This document is intended for Department MAILBOX (Manulife)'}, page_content='The lifecos have delivered strong total returns year to date (7.7% on average) versus the banks (2.1% on average) and the diversified financials covered by CIBC (2.9% on average). Lifeco valuations are no longer discounted, but we believe earnings growth and capital optionality favour the lifecos over banks. We are somewhat cautious heading into Q1 reporting due to the transition to IFRS 17 accounting and what we think are relatively high consensus expectations. However, we have a positive view on the sector for the next 12 to 24 months based on EPS growth, excess capital and potential for positive EPS revisions Exhibit 1: Canadi